In [ ]:
# Analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
pd.set_option("display.max_rows", None,"display.max_columns", None)
warnings.simplefilter(action='ignore')
plt.style.use('seaborn')

# Algorithmes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error

# Data Wrangling

In [ ]:
train_df=pd.read_excel('/kaggle/input/flight-price-prediction-dataset/Data_Train.xlsx')
train_df.head()

In [ ]:
test_df=pd.read_excel('/kaggle/input/flight-price-prediction-dataset/Test_set.xlsx')
test_df.head()

In [ ]:
train_df.shape

In [ ]:
test_df.shape

In [ ]:
train_df.info()

In [ ]:
test_df.info()

In [ ]:
#missing values
train_df.isna().sum()

In [ ]:
# drop missing values
train_df.dropna(inplace=True)

In [ ]:
#missing values
test_df.isna().sum()

In [ ]:
# check for duplication in train data
train_df.duplicated().sum()

In [ ]:
# drop duplication in train data
train_df.drop_duplicates(inplace=True)

In [ ]:
# check for duplication in test data
test_df.duplicated().sum()

In [ ]:
# drop duplication in test data
test_df.drop_duplicates(inplace=True)

In [ ]:
# concat train_df,test_df
df=pd.concat([train_df,test_df],ignore_index=True)
df.head()

In [ ]:
df.tail()

##### price column has null values because test data does not have price column

In [ ]:
df.Additional_Info.value_counts()

In [ ]:
df.head()

In [ ]:
#map string values in the total stops column into integer values
dic={'non-stop':0,
    '1 stop':1,
    '2 stops':2,
    '3 stops':3,
    '4 stops':4}

df['Total_Stops']=df['Total_Stops'].map(dic)
df['Total_Stops'].value_counts()

In [ ]:
#convert column Date_of_Journey from string type to datetime
df['Date_of_Journey']=pd.to_datetime(df['Date_of_Journey'])

In [ ]:
# create a column for the month
df['month']=df['Date_of_Journey'].dt.month.astype(str)
# the month type is integer but the month not quantitive value so I convert it to string 
df['day']=df['Date_of_Journey'].dt.day.astype(str)
# the month type is integer but the month not quantitive value so I convert it to string 

In [ ]:
df['Duration_hour']=df['Duration'].str.split('h').str[0]
df['Duration_min']=df['Duration'].str.split('h').str[1].replace(['m'],'')
#df['Duration_min']=df['Duration_min'].apply(lambda i:i[:-1]) it is not work
df['Duration_min']=df['Duration_min'].str[:-1]

In [ ]:
# those two rows make an error when I want to convert them to integer so I will drop it 
df[df['Duration_hour']=='5m']  

In [ ]:
df.drop(df[df['Duration_hour']=='5m'].index,inplace=True)

In [ ]:
df['Duration_hour']=df['Duration_hour'].astype(int)
#df['Duration_hour']=df['Duration_hour'].apply(lambda i: int(i))   #same result

In [ ]:
df['Duration_min'].replace('',np.nan,inplace=True)
df['Duration_min'].fillna(0,inplace=True)
df['Duration_min']=df['Duration_min'].astype(int)

In [ ]:
df.head()

In [ ]:
def convert_to_minutes(h):
    if h>12:
        return (h-12)*60
    else:
        return h*60
    
df['Duration_hour']=df['Duration_hour'].apply(convert_to_minutes)
df['Duration_min']=df['Duration_hour']+df['Duration_min']
df.drop(columns=['Duration_hour','Duration'],inplace=True)
df.head()

#### Arrival_time is mean the scheduled arrival time, and Dep_time is mean that the Departure departure time so we should not split them

In [ ]:
#df['Dep_hour']=df['Dep_Time'].str.split(':').str[0].astype(int)
#df['Dep_min']=df['Dep_Time'].str.split(':').str[1].astype(int)
#df.head()

# Time Series Analysis

##### what is the first date of the journey?

In [ ]:
df['Date_of_Journey'].max()

##### what is the last date of the journey?

In [ ]:
df['Date_of_Journey'].min()

In [ ]:
df['Arrival_Time'].str[:2].value_counts().plot(kind='line')
plt.xlabel('hours')
plt.ylabel('frequency')
plt.title("most hours of arrival time",
          fontsize='20',
          backgroundcolor='white',
          color='purple');

In [ ]:
df['Dep_Time'].str[:2].value_counts().plot(kind='line')
plt.xlabel('hours')
plt.ylabel('frequency')
plt.title("most hours of departure time",
          fontsize='20',
          backgroundcolor='white',
          color='purple');

In [ ]:
sns.lineplot(x='Date_of_Journey',y='Price',data=df[df['Price']!=np.nan],palette='crest')
plt.title("price of months ",
          fontsize='20',
          backgroundcolor='white',
          color='green');

##### January has most prices

In [ ]:
chart=sns.countplot(x='month',data=df,palette='crest', order = df['month'].value_counts().index)
plt.xlabel('months')
plt.ylabel('frequency')
plt.title("months frequency ",
          fontsize='20',
          backgroundcolor='white',
          color='green')
chart.set_xticklabels(chart.get_xticklabels(),rotation=360);

In [ ]:
sns.histplot(x='Duration_min',data=df,palette='crest')
plt.title("distribution of journey minuts",
          fontsize='20',
          backgroundcolor='white',
          color='green');

##### most journeys take 500 minutes

# EDA (Exploratory Data Analysis)

In [ ]:
plt.figure(figsize=(15,7))
chart=sns.countplot(x='Airline',data=df,palette='crest', order = df['Airline'].value_counts().index)
plt.title("airline frequency ",
          fontsize='20',
          backgroundcolor='white',
          color='green')
chart.set_xticklabels(chart.get_xticklabels(),rotation=45);

In [ ]:
fig,ax=plt.subplots(ncols=2,nrows=1,figsize=(15,5))
sns.countplot('Destination',data=df,palette='crest', order = df['Destination'].value_counts().index,ax=ax[0]).set(title='frequency of destination')
sns.countplot('Source',data=df,palette='crest', order = df['Source'].value_counts().index,ax=ax[1]).set(title='frequency of source')

plt.tight_layout();

##### cochin is the most destination from others
##### cochin is the most destination from others

In [ ]:
plt.figure(figsize=(15,7))
chart=sns.countplot(y='Route',data=df,palette='crest', order = df['Route'].value_counts()[:20].index)
plt.title("most 16 routes of journeys",
          fontsize='20',
          backgroundcolor='white',
          color='green');

In [ ]:
plt.figure(figsize=(15,7))
chart=sns.countplot(y='Route',data=df,palette='crest', order = df['Route'].value_counts()[:20].index)
plt.title("most 16 routes of journeys",
          fontsize='20',
          backgroundcolor='white',
          color='green');

In [ ]:
sns.countplot(x='Total_Stops',data=df,palette='crest', order = df['Total_Stops'].value_counts().index)
plt.title("total stops frequency ",
          fontsize='20',
          backgroundcolor='white',
          color='green');

##### most journeys take one stop during the journey

# Modeling

In [ ]:
#add column Len_Route insted of Route 
df['Len_Route']=df['Route'].apply(lambda i :len(i))
df.drop(columns=['Route'],inplace=True)

In [ ]:
# drop columns Date_of_Journey,Dep_Time and Arrival_Time
df.drop(columns=['Date_of_Journey','Dep_Time','Arrival_Time'],axis=1,inplace=True)

In [ ]:
plt.figure(figsize=(10,5))
correlation=df.corr()
sns.heatmap(correlation,vmin=None,
    vmax=0.8,
    cmap='crest',
    annot=True,
    fmt='.1f',
    linecolor='white',
    cbar=True);

In [ ]:
df.corr()['Price']

In [ ]:
#Convert categorical variable into dummy/indicator variables
df=pd.get_dummies(df)

In [ ]:
df.head()

In [ ]:
data=df[~(df['Price'].isna())]
data.tail()

In [ ]:
finalt_test_data=df[df['Price'].isna()]
finalt_test_data=finalt_test_data.drop(columns=['Price'])
finalt_test_data.head()

In [ ]:
X=data.drop(columns=['Price'])
y=data['Price']
X.shape,y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=24, shuffle =True)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

In [ ]:
CV = []
train_scores = []
test_scores = []

def pred_model(model):
    # Training model
    model.fit(X_train,y_train)
            
    # score of train set
    train_model_score = model.score(X_train,y_train)
    train_scores.append(round(train_model_score,2))
    y_pred_train=model.predict(X_train)
    
    # score of test set
    test_model_score = model.score(X_test,y_test)
    test_scores.append(round(test_model_score,2))
    y_pred_test=model.predict(X_test)
    
    # train set using Cross validation
    cross_val = cross_val_score(model ,X_train ,y_train ,cv=5)
    cv_mean = cross_val.mean()
    CV.append(round(cv_mean,2))
    
    # Printing results
    print("Train score :",round(train_model_score,2))
    print("Test score :",round(test_model_score,2))
    print("Train CV scores :",cross_val)
    print("Train CV mean :",round(cv_mean,2))
    print("Mean absolute error for train :",mean_absolute_error(y_train,y_pred_train))
    print("Median absolute error for train :",median_absolute_error(y_train,y_pred_train))
    print("Mean absolute error for test :",mean_absolute_error(y_test,y_pred_test))
    print("Median absolute error for test :",mean_absolute_error(y_test,y_pred_test))
    
    
    # Plotting Graphs 
    # Residual Plot of train data
    fig, ax = plt.subplots(1,2,figsize = (10,4))
    ax[0].set_title('Residual Plot of Train samples')
    sns.distplot((y_train[:100]-y_pred_train[:100]),hist = False,ax = ax[0])
    ax[0].set_xlabel('y_train - y_pred_train')
    
    # Y_test vs Y_train scatter plot
    ax[1].set_title('y_test vs y_pred_test')
    ax[1].scatter(x = y_test[:100], y = y_pred_test[:100])
    ax[1].set_xlabel('y_test')
    ax[1].set_ylabel('y_pred_test')
    
    plt.show()

#### Standard Linear Regression or Ordinary Least Squares

In [ ]:
lr = LinearRegression()
pred_model(lr)

#### SGDRegressor

In [ ]:
# Creating Ridge model object
SGDR = SGDRegressor()
 
dic={"penalty":['l2','l1', 'elasticnet'],
     "loss":['squared_loss','huber', 'epsilon_insensitive', 'squared_epsilon_insensitive']
    }

# Creating GridSearchCV to find the best estimator of hyperparameter
SGDR_rs = GridSearchCV(estimator = SGDR, param_grid=dic)
pred_model(SGDR_rs)

In [ ]:
SGDR_rs.best_estimator_

#### Random Forest

In [ ]:
RF = RandomForestRegressor(max_depth=50,min_samples_split=20,n_estimators=100)
pred_model(RF)

#### MLPRegressor

In [ ]:
MLPR=MLPRegressor(activation='logistic',solver='lbfgs')
pred_model(MLPR)

#### XGBRegressor

In [ ]:
XGBR = XGBRegressor()
pred_model(XGBR)

In [ ]:
Technique = ["LinearRegression","SGDRegressor","RandomForestRegressor","MLPRegressor","XGBRegressor"]
results=pd.DataFrame({'Model': Technique,'Accuracy(Train)': train_scores,'Accuracy(Test)': test_scores,
                      'CV score mean(Train)': CV})
display(results)

#### I will use the XGBoostRegressor algorithm on the second file test_set (finalt_test_data) to predict the output

In [ ]:
finalt_test_data.head()

In [ ]:
Prdicted_Price=XGBR.predict(finalt_test_data)
Prdicted_Price[:5]

In [ ]:
finalt_test_data['Prdicted_Price']=Prdicted_Price

In [ ]:
finalt_test_data.head()

#### thank you so much for reading to the end, your advice and comments are important to me. Don't forget to upvote this notebook if you like it!